# Workbench Quick Reference

Beyond the indexable interface, `GridWorkBench` provides
convenience methods for common tasks: data shortcuts, matrix
retrieval, voltage analysis, and case control.

```python
from esapp import GridWorkBench

wb = GridWorkBench("path/to/case.pwb")
```

In [ ]:
from esapp import GridWorkBench
from esapp.components import Bus, Gen
import numpy as np
import ast

with open('../../../examples/data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

wb = GridWorkBench(case_path)

## Data Shortcuts

These methods return DataFrames of common object types with their
most useful fields pre-selected.

| Method | Returns |
|---|---|
| `wb.gens()` | Generator MW, Mvar, and status |
| `wb.loads()` | Load MW, Mvar, and status |
| `wb.shunts()` | Switched shunt MW, Mvar, and status |
| `wb.lines()` | All transmission lines |
| `wb.transformers()` | All transformers |
| `wb.areas()` | All areas |
| `wb.zones()` | All zones |

In [ ]:
wb.gens().head()

In [ ]:
wb.loads().head()

## Power Flow

`pflow()` solves the AC power flow and returns complex bus voltages.
After solving, all data accessed through the indexable interface
reflects the updated system state.

```python
V = wb.pflow()                       # default: Polar Newton-Raphson
V = wb.pflow(method="POLARNEWT")     # explicit method selection
```

## Voltage & Mismatch

| Method | Returns |
|---|---|
| `wb.voltage()` | Complex bus voltages (per-unit) |
| `wb.voltage(complex=False)` | Magnitude and angle as separate Series |
| `wb.set_voltages(V)` | Set bus voltages from a complex array |
| `wb.mismatch()` | Bus active/reactive power mismatches |
| `wb.netinj()` | Net power injection at each bus |
| `wb.violations(v_min, v_max)` | Buses outside a voltage band |

In [ ]:
V = wb.voltage()
np.abs(V).describe()

## Matrix Access

| Method | Returns |
|---|---|
| `wb.ybus()` | System admittance matrix (sparse) |
| `wb.ybus(dense=True)` | Dense admittance matrix |
| `wb.jacobian()` | Power flow Jacobian |
| `wb.jacobian(form="P")` | Polar-form Jacobian |
| `wb.jacobian(form="DC")` | DC Jacobian |
| `wb.branch_admittance()` | Branch admittance matrices (Yf, Yt) |

In [ ]:
Y = wb.ybus()
Y.shape

In [ ]:
J = wb.jacobian()
J.shape

## Topology

| Method | Returns |
|---|---|
| `wb.busmap()` | Bus number → matrix index mapping |
| `wb.buscoords()` | Bus latitude/longitude from substation data |

In [ ]:
bmap = wb.busmap()
bmap.head()

## Case Control

| Method | Purpose |
|---|---|
| `wb.save()` | Save the case |
| `wb.save("new.pwb")` | Save to a new file |
| `wb.close()` | Close the current case |
| `wb.flatstart()` | Reset to 1.0 pu / 0° |
| `wb.edit_mode()` | Enter edit mode |
| `wb.run_mode()` | Enter run mode |
| `wb.command(script)` | Execute a raw script command |
| `wb.log(msg)` | Add a message to the PowerWorld log |
| `wb.print_log()` | Print the message log |

## Branch Switching

Open or close branches by specifying bus numbers and circuit ID.

```python
wb.open_branch(1, 2, ckt='1')   # open the branch
wb.close_branch(1, 2, ckt='1')  # close it back
```

## Utility Modules

GridWorkBench embeds specialized analysis modules accessible as
attributes:

| Attribute | Module | Capabilities |
|---|---|---|
| `wb.network` | Network topology | Incidence matrices, Laplacians, bus coordinates |
| `wb.gic` | GIC analysis | G-matrix, E-field Jacobians, storm simulation |

See the :doc:`API Reference </api/utils>` for full documentation.